Este projeto é sobre previsão de rendimento de mirtilo.

O objetivo é prever o *target* ***yield***.

**Atualizações:**
versão 6: atualizei alguns códigos com objetivo de otimização de tempo de execução e uso de memória.

*Nas versões anteriores eu havia utilizado* ***Linear Regression*** *como método de previsão, tive um* ***MAE: 368.19*** *e* ***R2: 0.8138***.

*Porém olhando o notebook do melhor score desse dataset no* ***Kaggle***, *eu me deparei com o* ***LGBM***, *que é um método que utiliza uma árvore de decisão base e várias outras para reduzir os erros das anteriores.*

*Como esse notebook se trata de um estudo, eu resolvi utilizar esse método para testar como ele funciona. Cheguei a um resultado de* ***MAE: 340.13*** *e* ***R2: 0.8231***.

Inicialmente, vamos importar todas as bibliotecas necessárias para a realização desse projeto.

In [ ]:
# Istalação de bibliotecas

!pip install dtype_diet

In [ ]:
# importação das bibliotecas e ferramentas de ML e de métricas
# manipulação de dados
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from dtype_diet import report_on_dataframe, optimize_dtypes

# visualização de dados
import matplotlib.pyplot as plt
import seaborn as sns

# pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import FunctionTransformer, StandardScaler
from category_encoders import TargetEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline

# machine learning
import lightgbm as lgb

# métricas
from sklearn.metrics import r2_score, mean_absolute_error, mean_squared_error

Nessa etapa, iremos importar o *dataset* da Kaggle que está em formato CSV.

Armazenaremos ele na variável **'df'** utilizando o modelo de *DataFrame* do **Pandas**.

E por fim, faremos uma visualização do cabeçalho.

In [ ]:
# importação do dataframe de treino armazenado no kaggle
df = pd.read_csv('/kaggle/input/playground-series-s3e14/train.csv')

# visualização de parte dos dados
df.head()

No comando abaixo poderemos ver um resumo de informações, como a quantidade de colunas, quantidade de dados *non-null* e o ***Dtype***.

In [ ]:
# resumo do tipo de dados de cada coluna e a quantidade de dados para cada coluna
df.info()

Com objetivo de otimizar o uso de memória pelo *DataFrame* eu utilizei uma biblioteca chamada ***dtype_diet***.

Essa biblioteca tem uma função chamada ***report_on_dataframe()*** que entrega um *report* em *DataFrame* com os tipos de dados utilizados (normalmente o padrão) e uma sujestão de tipos de dados mais adequados que consomem menos memória.

Dentro da própria biblioteca existe uma outra função para fazer a alterção automática desses tipos de dados chamada ***optimize_dtypes()***.

Como podem ver abaixo, tivemos uma redução de aproximadamente **8%** do consumo de memória. Pode não parecer muito, porque nesse caso os tipos de dados padrões já estavam relativamente adequados, mas existem *DataFrames* que eu consegui reduzir cerca de **95%** o consumo de memória. 

Imagine uma base de dados com **2GB ser reduzida para 100MB**... Isso te coloca em outro patamar, meu amigo!

In [ ]:
# Variável com o uso de memória do DF original já convertido para MB
original_memory = df.memory_usage(deep=True).sum()/1024/1024

# Criando DF com os tipos de dados e possiveis otmizações
df_report = report_on_dataframe(df)

# Substituindo o DF antigo pelo novo DF otimizado
df = optimize_dtypes(df, df_report)

# Variável com o uso de memória do novo DF já convertido para MB
new_memory = df.memory_usage(deep=True).sum()/1024/1024

# Visualização do consumo e da redução do consumo
print(f'Original df memory: {original_memory} MB')
print(f'New df memory: {new_memory} MB')
print(f'Reduction: {(1 - (new_memory / original_memory)) * 100:.02f}%')

Analisando o cabeçalho já conseguimos identificar que todos os dados são numéricos, sendo quase todos contínuos. Apenas a *feature* ***'RainingDays'*** que parece ser discreto.

Para termos mais informações, utilizaremos a sumarização a seguir:

In [ ]:
# sumarização dos dados para encontrar outliers ou anomalias
df.describe()

Aparentemente os dados parecem normais, com excessão da *feature* ***'honeybee'***, onde podemos ver que o valor máximo dela é muito distante dos demais valores.

Trataremos isso mais adiante na construção dos *pipelines*.

A seguir, faremos uma contagem de quantos *outliers* temos na feature ***honeybee***.

In [ ]:
# contagem de quantos outliers existem na feature 'honeybee'
mediana = df['honeybee'].median()
dpad = df['honeybee'].std()
filtro = mediana + (2 * dpad)
print(f'{df["honeybee"][df["honeybee"] > filtro].count()} Outliers')

Agora, para termos uma ideia de quais *features* tem mais relevância no *target*, faremos uma *list comprehension* (até a versão 5 eu havia utilizado laço de repetição) para ver a correlação de cada *feature* com o nosso *target*.

A vantagem da *list comprehension* é que ela executa muito mais rápido em diversos casos.

Na versão 5 eu criei um *for in* com *if* para visualizar a correlação de cada variável independente em relação a variável dependente e tive um tempo de execução de **12.6 ms**, já com a *list comprehension* consegui o mesmo resultado com apenas **6.11 ms**.

Lembrando novamente, esses tempos são pequenos, pois essa base é pequena. Mas isso é uma ótima pratica quando se utiliza de bases gigantescas.

In [ ]:
#%%timeit # utilizado para ver o tempo gasto para executar o bloco
# Visualização da correlação de cada variável independente em relação à variável dependente

exclusion = ['id','yield']
corr_rounded = pd.DataFrame()
corr_rounded[0] = pd.DataFrame([c for c in df.columns if c not in exclusion])
corr_rounded[1] = round(pd.DataFrame([df[c].corr(df['yield']) for c in df.columns if c not in exclusion]), 3)
corr_rounded

Como vimos acima, as três últimas *features* tem uma correlação forte com o *target*.

Abaixo, iremos plotar essas três *features* em um gráfico *scatter* para vermos a distribuição dos dados nele.

In [ ]:
# plotagem das três maiores correlações com o yield
fig, axs = plt.subplots(nrows=2, ncols=3,figsize=(20, 10))

# Criando uma coluna no DataFrame para indicar a cor dos pontos com base na condição
color = np.where(df['RainingDays'] > df['RainingDays'].mean(),'maior','menor/igual')
axs = axs.flatten()

for i in range(3):
    sns.scatterplot(x=df.iloc[:, i + 14], y=df['yield'],hue=color, ax=axs[i], palette=['blue','orange'])
    axs[i].set_xlabel(df.columns[i + 14])
    axs[i].set_ylabel('Yield')

for i in range(3,6):
    sns.scatterplot(x=df.iloc[:, i + 8], y=df['yield'],hue=color, ax=axs[i], palette=['blue','orange'])
    axs[i].set_xlabel(df.columns[i + 8])
    axs[i].set_ylabel('Yield')

plt.show()

Podemos ver nos gráficos, que existe uma tendência de "desfunilamento". Quero dizer que, quanto maior o valor da *feature*, maior é o valor do *target* porém com uma amplitude maior também.

Já da para termos uma ideia de que nosso modelo provavelmente irá errar mais na predição dos *targets* maiores.

A partir de agora, iniciaremos nosso *pipeline*.

Na versão 5, eu usei o *for in* com dois *if* e comecei pelas *features* numéricas, dessa vez faremos de forma mais eficiente.

Abaixo, começaremos criando uma *list comprehension* com todas as *features* e declaramos qual o *target*.

*Detalhe: Como temos apenas features numéricas, eu poderia ter feito de uma forma mais simples, apenas removendo as colunas 'id' e 'yield', porém preferi manter esse esquema, pois essa é uma forma automatizada  e de boa prática de criar a lista. Nesse caso não foi necessariamente mais eficiente, mas em outros casos pode ser*.

In [ ]:
#%%timeit # utilizado para ver o tempo gasto para executar o bloco
# Selecionando features e target

features = [c for c in df.columns if c not in ['id', 'yield']]

target = [
    'yield'
]

features

In [ ]:
# %%timeit # utilizado para ver o tempo gasto para executar o bloco
# Início do pipeline de treinamento
# Criação de lista para numerical e Categorical features

numerical_features = [c for c in df.columns if c != 'id' and c != 'yield' if df[c].dtype != 'object']
categorical_features = list(set(features) - set(numerical_features))

print("Numerical Features:", numerical_features)
print("Categorical Features:", categorical_features)

Agora faremos a separação dos dados de treino e teste.

In [ ]:
# separação de dados treino e teste
X = df[features]
y = df[target]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

Com a separação realizada, podemos começar a escrever o *pipeline* de preprocessamento. 

Como temos apenas dados numéricos, faremos o preprocessamento apenas para ele.

Suavizaremos os *outliers* utilizando logarítimo.

Utilizaremos o método de padronização escalar e substituição dos dados nulos pela mediana do conjunto.

Aplicaremos o modelo de **regressão LGBM** e depois o *fit* (treino) do modelo.

In [ ]:
# Preprocessamento de outliers de colunas numéricas
outliers = Pipeline([
    ('transformer', FunctionTransformer(np.log1p))
])

# Preprocessamento de colunas numéricas
numeric_transformer = Pipeline([
    ('imputer', SimpleImputer(strategy='median')),
    ('scaler', StandardScaler())
])

# Combinando pré-processadores de colunas numéricas e categóricas
preprocessor = ColumnTransformer([
    ('outliers', outliers, numerical_features),
    ('numeric', numeric_transformer, numerical_features)
])

# Criando o pipeline com etapas de pré-processamento e modelo
pipeline = Pipeline([
    ('preprocessor', preprocessor),
    ('model', lgb.LGBMRegressor(
        objective= "regression_l1",
        metric= "mae"
))
])

# Resetando o índice
X_train.reset_index(drop=True, inplace=True)
y_train.reset_index(drop=True, inplace=True)

# Treinando o pipeline
pipeline.fit(X_train, y_train.values.ravel())

Após o treinamento, faremos a previsão dos dados de teste.

In [ ]:
# Fazendo predição com o modelo treinado no pipeline
y_pred = pipeline.predict(X_test)

E por fim, a visualização dos resultados.

In [ ]:
# visualização dos resultados obtidos
print(f'R2: {r2_score(y_test, y_pred)} --> Quantos % a predição representa o resultado real')
print(f'MAE: {mean_absolute_error(y_test, y_pred)} --> Erro médio absoluto')
print(f'MSE: {mean_squared_error(y_test, y_pred)} --> Erro médio quadrático')